In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
name_df = "train_claud_2_0"

train_df = f'/content/drive/MyDrive/data/{name_df}.csv'

train_df = pd.read_csv(train_df)

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158728 entries, 0 to 158727
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             158728 non-null  object
 1   comment_text   158728 non-null  object
 2   toxic          158728 non-null  int64 
 3   severe_toxic   158728 non-null  int64 
 4   obscene        158728 non-null  int64 
 5   threat         158728 non-null  int64 
 6   insult         158728 non-null  int64 
 7   identity_hate  158728 non-null  int64 
 8   token_count    158728 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 10.9+ MB


# Цей код реалізує повний процес навчання та використання моделі класифікації токсичності на основі BERT. Ось короткий опис основних компонентів:

Завантаження та підготовка даних
Створення користувацького класу Dataset для роботи з даними
Ініціалізація токенізатора BERT та моделі для класифікації послідовностей
Створення DataLoader для ефективного завантаження даних
Налаштування оптимізатора та функції втрат
Функції для навчання та оцінки моделі
Цикл навчання моделі
Збереження навченої моделі
Функція для прогнозування токсичності нових коментарів

Цей код надає вам основу для роботи з проектом. Ви можете адаптувати його відповідно до ваших конкретних потреб, наприклад, змінити гіперпараметри, додати валідацію або реалізувати додаткові функції.
Чи хотіли б ви, щоб я детальніше пояснив якусь частину коду або надав додаткову інформацію про певні аспекти проекту?

In [6]:
import torch
import os
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

# Завантаження та підготовка даних
df = train_df
texts = df['comment_text'].values
labels = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

# Розділення на тренувальний та тестовий набори
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Клас для створення набору даних
class ToxicityDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(label)
        }

# Ініціалізація токенізатора та моделі
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

# Створення наборів даних
train_dataset = ToxicityDataset(train_texts, train_labels, tokenizer, max_len=256)
val_dataset = ToxicityDataset(val_texts, val_labels, tokenizer, max_len=256)

# Створення завантажувачів даних
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Налаштування оптимізатора та функції втрат
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.BCEWithLogitsLoss()

# Функція для навчання моделі
def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model.train()
    losses = []
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return np.mean(losses)

# Функція для оцінки моделі
def eval_model(model, data_loader, loss_fn, device):
    model.eval()
    losses = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
            losses.append(loss.item())

    return np.mean(losses)

# Навчання моделі
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 3
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device)
    print(f'Train loss: {train_loss}')
    val_loss = eval_model(model, val_loader, loss_fn, device)
    print(f'Val loss: {val_loss}')
    print('-' * 10)


# Створіть папку для збереження моделі, якщо вона ще не існує
save_path = '/content/drive/My Drive/toxicity_model'
os.makedirs(save_path, exist_ok=True)

# Збереження моделі
torch.save(model.state_dict(), os.path.join(save_path, 'toxicity_model.pth'))
print(f"Модель збережено на Google Drive у папці: {save_path}")

# # Додайте код для завантаження моделі (для майбутнього використання)
# def load_model(model, path):
#     model.load_state_dict(torch.load(path))
#     return model

# Приклад завантаження моделі
# loaded_model = load_model(BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6), os.path.join(save_path, 'toxicity_model.pth'))

# Функція для прогнозування токсичності
def predict_toxicity(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=256,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.sigmoid(outputs.logits)

    return predictions.cpu().numpy()[0]

# # Приклад використання
# sample_text = "You are an idiot!"
# toxicity_scores = predict_toxicity(sample_text)
# print(f"Токсичність коментаря '{sample_text}':")
# print(f"Toxic: {toxicity_scores[0]:.4f}")
# print(f"Severe Toxic: {toxicity_scores[1]:.4f}")
# print(f"Obscene: {toxicity_scores[2]:.4f}")
# print(f"Threat: {toxicity_scores[3]:.4f}")
# print(f"Insult: {toxicity_scores[4]:.4f}")
# print(f"Identity Hate: {toxicity_scores[5]:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
Train loss: 0.05264688588394149
Val loss: 0.040243079591167796
----------
Epoch 2/3
Train loss: 0.0356082279573914
Val loss: 0.038559928578473505
----------
Epoch 3/3
Train loss: 0.02784090594916476
Val loss: 0.044184952334738604
----------
Модель збережено на Google Drive у папці: /content/drive/My Drive/toxicity_model
